# EfficientNet

## Installation and import of the libraries

In [1]:
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install tensorflow
%pip install efficientnet
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import efficientnet.tfkeras as efn
from tensorflow.keras.preprocessing import image



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


2024-11-17 07:36:20.586816: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 07:36:20.690957: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 07:36:20.777339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731825380.851304  210051 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731825380.874144  210051 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-17 07:36:21.062229: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## Preprocess

In [ ]:
dtypes = {
    'cane_height_type': pd.Int64Dtype(),
    'closure_placement': pd.Int64Dtype(),
    'heel_shape_type': pd.Int64Dtype(),
    'knit_structure': pd.Int64Dtype(),
    'length_type': pd.Int64Dtype(),
    'neck_lapel_type': pd.Int64Dtype(),
    'silhouette_type': pd.Int64Dtype(),
    'sleeve_length_type': pd.Int64Dtype(),
    'toecap_type': pd.Int64Dtype(),
    'waist_type': pd.Int64Dtype(),
    'woven_structure': pd.Int64Dtype()
}
df = pd.read_csv("./archive/attribute_data_2.csv", dtype=dtypes)
product_data_df = pd.read_csv('./archive/product_data.csv')
df = df.merge(product_data_df[['cod_modelo_color', 'des_filename']], on='cod_modelo_color', how='left')
# img_dict = dict(zip(product_data_df['cod_modelo_color'], product_data_df['des_filename']))
# Assume the CSV has columns ['image_id', 'attribute', 'value']
# One-hot encode the 'value' column for each attribute
attribute_columns = ['silhouette_type', 'neck_lapel_type', 'woven_structure', 'knit_structure',
                     'heel_shape_type', 'length_type', 'sleeve_length_type', 'toecap_type',
                     'waist_type', 'closure_placement', 'cane_height_type']

base_path = "./archive/images/images/"
df = df[df['des_filename'].apply(lambda x: os.path.exists(os.path.join(base_path, x)))]
df.reset_index(drop=True, inplace=True)
# df = df.head(1000)
# df = df.sample(n=10000, random_state=42)
unique_values_dict = {col: df.drop(columns=['cod_modelo_color', 'des_filename'])[col].dropna().unique().tolist() for col in attribute_columns}
mapping_dict = {col: {value: idx+1 for idx, value in enumerate(unique_values_dict[col])} 
                for col in unique_values_dict}

In [3]:

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

def load_and_preprocess_image(image_path, label):
    # Load and decode the image file
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode the JPEG image (if you have other formats, use decode_image)

    # Resize the image
    img = tf.image.resize(img, [224, 224])

    # Normalize the image
    img = img / 255.0  # Normalize to [0, 1]

    # Set the shape explicitly
    img.set_shape([224, 224, 3])

    return img, label

def one_hot_encode(value, possible_values, max_length):
    # Create a one-hot encoded vector
    encoding = [1 if val == value else 0 for val in possible_values]
    return encoding + [0] * (max_length - len(encoding))  # Pad with zeros if needed

def one_hot_encode_labels(labels):
    labels = labels.apply(lambda col: col.apply(lambda value: mapping_dict[col.name].get(value, value)))
    final_array = []
    max_length = max([len(list(values.values())) for col, values in mapping_dict.items()])
    for _, row in labels.iterrows():
        one_hotted_row = []
        for col_name in labels.columns:
            possible_values = list(mapping_dict[col_name].values()) if col_name in mapping_dict else [0]  # Handle 0 values
            one_hotted_row.append(one_hot_encode(row[col_name], possible_values, max_length))
        final_array.append(one_hotted_row)
    final_array
    return final_array

def create_dataset(df, image_folder, batch_size=24):
    # Convert filenames and labels into tensors
    image_paths = df['des_filename'].apply(lambda x: os.path.join(image_folder, x)).values
    labels = df.drop(columns=['cod_modelo_color', 'des_filename']).fillna(0)  # Replace NaNs with -1 if necessary
    labels = one_hot_encode_labels(labels)
    # Convert to TensorFlow tensors
    image_paths = tf.convert_to_tensor(image_paths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)

    # Create a tf.data.Dataset from image paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Apply the preprocessing function to load and preprocess images
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    # Batch the data and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

# Create the train and test datasets
train_dataset = create_dataset(train_df, './archive/images/images/')
test_dataset = create_dataset(test_df, './archive/images/images/')

2024-11-17 07:36:27.250314: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## Build

In [4]:
# Build the EfficientNet model
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Unfreeze only the last 10 layers
    layer.trainable = False

# Create the model architecture
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)

# Number of attributes (11 in your case) and number of possible values (29 for each)
num_attributes = 11
num_possible_values = 33

# Create the output layers
outputs = []
for _ in range(num_attributes):  # 11 output layers, one for each attribute
    outputs.append(layers.Dense(num_possible_values, activation='softmax')(x))

# Stack all the output layers into one model output
outputs = layers.concatenate(outputs, axis=-1)  # Concatenate along the last axis
outputs = layers.Reshape((num_attributes, num_possible_values))(outputs)


# Build and compile the model
model = models.Model(inputs, outputs)

# Compile with a loss function for multi-output classification
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnet-b0     │ (None, 7, 7,      │  4,049,564 │ input_layer_1[0]… │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnet-b0[… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 33)        │     42,273 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 363)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0],    │
│                     │                   │            │ dense_3[0][0],    │
│                     │                   │            │ dense_4[0][0],    │
│                     │                   │            │ dense_5[0][0],    │
│                     │                   │            │ dense_6[0][0],    │
│                     │                   │            │ dense_7[0][0],    │
│                     │                   │            │ dense_8[0][0],    │
│                     │                   │            │ dense_9[0][0],    │
│                     │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 11, 33)    │          0 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,514,567 (17.22 MB)

 Trainable params: 1,358,235 (5.18 MB)

 Non-trainable params: 3,156,332 (12.04 MB)

## Train

In [5]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

Epoch 1/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 179s 515ms/step - accuracy: 0.4284 - loss: 0.6610 - val_accuracy: 0.5055 - val_loss: 0.4705
Epoch 2/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 169s 507ms/step - accuracy: 0.5002 - loss: 0.4416 - val_accuracy: 0.5040 - val_loss: 0.4497
Epoch 3/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 170s 510ms/step - accuracy: 0.5177 - loss: 0.3891 - val_accuracy: 0.4979 - val_loss: 0.4490
Epoch 4/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 171s 512ms/step - accuracy: 0.5189 - loss: 0.3520 - val_accuracy: 0.5040 - val_loss: 0.4587
Epoch 5/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 173s 518ms/step - accuracy: 0.5227 - loss: 0.3105 - val_accuracy: 0.5056 - val_loss: 0.4798
Epoch 6/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 171s 513ms/step - accuracy: 0.5406 - loss: 0.2721 - val_accuracy: 0.4853 - val_loss: 0.5080
Epoch 7/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 171s 513ms/step - accuracy: 0.5597 - loss: 0.2377 - val_accuracy: 0.4949 - val_loss: 0.5282
Epoch 8/10
334/334 ━━━━━━━━━━━━━━━━━━━━ 173s 518ms/step - accuracy: 0.5704 -

## Test

In [51]:
test_df = pd.read_csv("./archive/test_data.csv", dtype=dtypes)[['cod_modelo_color', 'des_filename']]
# test_df = test_df.sample(n=35000, random_state=42).reset_index(drop=True)


In [52]:

# List to collect IDs of images that do not exist
to_remove_ids = []

# Function to preprocess images, similar to the training pipeline
def load_and_preprocess_image(image_path):
    # Read and decode the image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Decode image to RGB
    img = tf.image.resize(img, [224, 224])  # Resize image to 224x224
    img = img / 255.0  # Normalize the image to [0, 1]

    img.set_shape([224, 224, 3])  # Explicitly set shape to avoid shape issues
    return img

# Function to make predictions in batches
def predict_in_batches(df, image_folder, model, batch_size=50):
    # Extract and filter image paths from the dataframe
    image_paths = df['des_filename'].apply(lambda x: os.path.join(image_folder, x)).values
    valid_image_paths = [path for path in image_paths if os.path.exists(path)]
    to_remove_ids.extend([path for path in image_paths if not os.path.exists(path)])

    # Create a TensorFlow Dataset from valid image paths
    dataset = tf.data.Dataset.from_tensor_slices(valid_image_paths)

    # Map the image paths to their respective preprocessed images
    dataset = dataset.map(lambda path: load_and_preprocess_image(path), num_parallel_calls=tf.data.AUTOTUNE)

    # Batch the dataset and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    # Initialize an empty list to store predictions
    predictions = []

    # Loop through the dataset and make predictions
    for batch_images in dataset:
        batch_predictions = model.predict(batch_images)
        predictions.extend(batch_predictions)  # Collect all predictions

    # Return predictions and the list of missing file paths
    return predictions, to_remove_ids

# Predict on the batch of images
predictions, to_remove_ids = predict_in_batches(test_df, './archive/images/images/', model)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 275ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 300ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 282ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 255ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 255ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 267ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 282ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 279ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 

2024-11-17 09:23:16.240370: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [54]:
test_df = test_df[test_df['des_filename'] != '88_49720742_67044470-56_.jpg']

In [53]:
to_remove_ids

['./archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg',
 './archive/images/images/88_49720742_67044470-56_.jpg']

In [55]:
prediction_mapped = []
for prediction in predictions:
    labels_mapped = []
    for i, label in enumerate(prediction):  # Use enumerate to track index in prediction
        max_idx = np.argmax(label)  # Find the index of the max value in the label
        if label[max_idx] > 0.85:
            labels_mapped.append(int(max_idx))  # Set the value at the current position to max_idx
        else:
            labels_mapped.append(int(0))
    prediction_mapped.append(labels_mapped)
prediction_mapped
predictions_df = pd.DataFrame(prediction_mapped, columns=attribute_columns)

In [56]:
predictions_df['cod_modelo_color'] = test_df['cod_modelo_color']

In [57]:
attributes_df = pd.read_csv("./archive/attribute_data.csv", dtype=dtypes)
# Assuming attributes_df is your DataFrame
attributes_dict = {}

# Iterate over each row and create the key-value pairs
for _, row in attributes_df.iterrows():
    key = f"{row['attribute_name']}_{row['cod_value']}"
    value = row['des_value']
    attributes_dict[key] = value

In [63]:
melted_df = predictions_df.melt(id_vars=['cod_modelo_color'], var_name='attribute', value_name='value')
# melted_df = melted_df[melted_df['value'] != 0]

In [64]:
melted_df['test_id'] = melted_df['cod_modelo_color'] + '_' + melted_df['attribute'].astype(str)
melted_df['des_value'] = melted_df['attribute'] + '_' + melted_df['value'].astype(str)
melted_df['des_value'] = melted_df['des_value'].apply(lambda x: attributes_dict.get(x, 'INVALID'))

melted_df = melted_df.drop(columns=['cod_modelo_color', 'attribute', 'value'])
melted_df = melted_df.drop_duplicates(subset='test_id', keep='first')
melted_df.to_csv('archive/submission.csv', index=False)

In [66]:
sample_df = pd.read_csv("./archive/sample_submission.csv")
difference_df = sample_df.merge(melted_df, on='test_id', how='left', indicator=True)
values_not_in_melted_df = difference_df[difference_df['_merge'] == 'left_only']

# Extract the values you want
values_not_in_melted_df

,test_id,des_value_x,des_value_y,_merge
5771,88_49720742_silhouette_type,Caja,NaN,left_only
12205,88_49720742_neck_lapel_type,Recto,NaN,left_only
18853,88_49720742_length_type,Halter,NaN,left_only
25260,88_49720742_sleeve_length_type,Alta,NaN,left_only
32300,88_49720742_woven_structure,Maxi,NaN,left_only
38764,88_49720742_closure_placement,Palabra Honor,NaN,left_only
45055,88_49720742_cane_height_type,Midi,NaN,left_only
51606,88_49720742_heel_shape_type,Cargo,NaN,left_only
58190,88_49720742_knit_structure,Capucha,NaN,left_only
64734,88_49720742_waist_type,Cuadrada,NaN,left_only


In [68]:
# Find values in melted_df but not in sample_df using the test_id column
difference_df = melted_df.merge(sample_df, on='test_id', how='left', indicator=True)
values_not_in_sample_df = difference_df[difference_df['_merge'] == 'left_only']

# Extract the values you want
values_not_in_sample_df

,test_id,des_value_x,des_value_y,_merge
4119,NaN,INVALID,NaN,left_only


In [381]:
predictions_df

,silhouette_type,neck_lapel_type,woven_structure,knit_structure,heel_shape_type,length_type,sleeve_length_type,toecap_type,waist_type,closure_placement,cane_height_type,cod_modelo_color
0,0,0,1,4,0,0,0,0,0,0,1,84_1178361
1,0,0,0,4,2,0,0,0,2,0,1,82_1087439
2,0,3,0,0,0,0,0,0,2,0,0,84_1173610
3,0,0,0,4,0,15,0,0,0,0,0,82_1091921
4,0,3,0,0,2,0,0,0,2,0,0,81_1049388
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,0,0,4,0,0,0,0,2,0,0,84_1166571
96,0,0,0,0,0,0,0,0,0,0,1,82_1100151
97,0,0,0,0,0,0,0,0,2,0,1,82_1108057
98,0,0,0,4,0,0,0,0,2,0,0,86_1223385


## Submission

In [ ]:
attributes_df = pd.read_csv("./archive/attribute_data.csv", dtype=dtypes)
# Assuming attributes_df is your DataFrame
attributes_dict = {}

# Iterate over each row and create the key-value pairs
for _, row in attributes_df.iterrows():
    key = f"{row['attribute_name']}_{row['cod_value']}"
    value = row['des_value']
    attributes_dict[key] = value

In [336]:
predictions_df

,silhouette_type_1,silhouette_type_2,silhouette_type_3,silhouette_type_4,silhouette_type_8,silhouette_type_9,silhouette_type_12,silhouette_type_14,silhouette_type_17,silhouette_type_20,...,closure_placement_1,closure_placement_2,closure_placement_3,closure_placement_4,closure_placement_5,closure_placement_6,cane_height_type_1,cane_height_type_2,cane_height_type_3,cod_modelo_color
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,83_1127226
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,83_1145464
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,84_1156442
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,83_1132778
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,85_1210126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,85_1186056
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,86_1229520
97,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,84_1186422
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,84_1165349


In [62]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 11), dtype=tf.int32, name=None))>

In [70]:
model.save("/home/gerard/Desktop/my_model.keras")

In [163]:
columns_to_exclude = ['cod_modelo_color', 'des_filename']
unique_values_dict = {}
for column in df.columns:
    if column not in columns_to_exclude:
        # Get unique values excluding NaN
        unique_values_dict[column] = df[column].dropna().unique().tolist()
columns = ['silhouette_type', 'neck_lapel_type', 'woven_structure',
       'knit_structure', 'heel_shape_type', 'length_type',
       'sleeve_length_type', 'toecap_type', 'waist_type',
       'closure_placement', 'cane_height_type']
unique_values_dict

{'cane_height_type': [3, 2, 1],
 'closure_placement': [1, 4, 3, 6, 5, 2],
 'heel_shape_type': [1, 7, 5, 6, 8, 4, 2, 13],
 'knit_structure': [3, 1, 2, 4],
 'length_type': [3, 5, 10, 8, 9, 4, 11, 13, 2, 7, 6, 1],
 'neck_lapel_type': [6,
  31,
  25,
  5,
  7,
  13,
  24,
  15,
  22,
  19,
  26,
  21,
  14,
  1,
  17,
  32,
  10,
  23,
  33,
  18,
  11,
  3,
  29,
  12,
  9,
  20,
  16,
  2,
  4],
 'silhouette_type': [32,
  27,
  20,
  37,
  14,
  34,
  17,
  44,
  3,
  1,
  12,
  47,
  38,
  45,
  36,
  2,
  31,
  26,
  23,
  42,
  21,
  28,
  48],
 'sleeve_length_type': [2, 1, 5, 3, 4, 6],
 'toecap_type': [2, 3, 4],
 'waist_type': [1, 4, 2, 3],
 'woven_structure': [3, 1, 2, 4]}